<a href="https://colab.research.google.com/github/Alanawd/AIDS/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenize

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
encoded_input = tokenizer(["Hello, I'm a single sentence!", "I'm a second sentence", "I loved the show!"], padding=True, return_tensors='pt')
encoded_input

In [ ]:
tokenizer.decode(encoded_input['input_ids'][2])

In [ ]:
encoded_input['input_ids'][1][encoded_input['attention_mask'][1]==1]

In [ ]:
tokenizer.convert_ids_to_tokens(encoded_input["input_ids"][0])

# Load dataset

In [ ]:
import transformers
from datasets import list_datasets


In [ ]:
datasets_list = list_datasets()
len(datasets_list)
print(', '.join(dataset for dataset in datasets_list))
# https://huggingface.co/datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset('trec')

In [ ]:
dataset

In [ ]:
dataset['train'].shuffle()[:5]

In [ ]:
dataset['train'].features

In [ ]:
dataset['train'].features['label-coarse'].names

In [ ]:
dataset = dataset.remove_columns("label-fine")
dataset = dataset.rename_column('label-coarse', 'labels')

In [ ]:
dataset['train'].features

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
train_dataset = tokenized_datasets['train']
small_train_dataset = train_dataset.shuffle().select(range(500))
eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6, n_layers=3)

In [ ]:
model.to('cuda')

In [ ]:
model.config.n_layers

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("uni_twitter/bert_trainer/",
                                  evaluation_strategy="epoch", 
                                  num_train_epochs = 10,
                                  per_device_train_batch_size=32)

In [ ]:
training_args

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=eval_dataset
)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model.device

In [ ]:
sentence = 'Where are my keys?'
tokens = tokenizer(sentence, return_tensors='pt')
tokens
# tokenizer.decode(tokens['input_ids']), tokens
preds = model(tokens['input_ids'].to(model.device))[0]
idx = preds.argmax(-1)

small_train_dataset.features['labels'].names[idx]

for a,b in zip(small_train_dataset.features['labels'].names, preds.softmax(-1).detach().cpu().numpy()[0]):
    print(a, b)
    

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics)
trainer.train()

In [ ]:
sentence = 'Where are my keys?'
tokens = tokenizer(sentence, return_tensors='pt')
tokens
# tokenizer.decode(tokens['input_ids']), tokens
preds = model(tokens['input_ids'].to(model.device))[0]
idx = preds.argmax(-1)

small_train_dataset.features['labels'].names[idx]

for a,b in zip(small_train_dataset.features['labels'].names, preds.softmax(-1).detach().cpu().numpy()[0]):
    print(a, b)

In [ ]:
trainer.evaluate()

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    
    inputs = tokenizer(text, return_tensors='pt')
    # perform inference to our model
    outputs = model(inputs['input_ids'].to(model.device))
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return small_train_dataset.features['labels'].names[probs.argmax()], probs.max().detach().cpu().item()

In [ ]:
get_prediction(['How are you?'])

# GPT-2 Text Generation

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
tokenized = tokenizer("what are you going to do?", return_tensors='pt')
tokenized

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id = tokenizer.eos_token_id)
model.to('cuda')

In [ ]:
ids = tokenizer("The shawshank")['input_ids']
[tokenizer.decode(x) for x in ids]

In [ ]:
text = "I lost my keys"
input_ids = tokenizer.encode(text, return_tensors='pt')
# tokenizer.convert_ids_to_tokens(input_ids[0])
input_ids

In [ ]:
output = model.generate(input_ids.to(model.device), 
max_length = 10000, 
num_beams = 10,
no_repeat_ngram_size  = 2)

In [ ]:
tokenizer.decode(output.squeeze(0))